### Imports

In [151]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")


In [152]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [153]:
dataset = pd.read_csv('diabetes-dataset.zip')

Viewing dataset

In [154]:
def about_dataset(df):
    print('*******SHAPE*********')
    print('Shape of the dataset is : ',df.shape)
    print('*******Head*********')
    print(df.head())
    print('*******Tail*********')
    print(df.tail())
    print('*******Columns in the dataframe*********')  
    print(df.columns)
    print('*******About the dataset*********')
    print(df.describe())

In [155]:
about_dataset(dataset)

*******SHAPE*********
Shape of the dataset is :  (768, 9)
*******Head*********
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  
*******Tail*********
     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
763           10      101             76             48      180  32.9   
764            2      122             70             27

Split train-test

70% train 
30% test

In [156]:
y = dataset["Outcome"]
X = dataset.drop("Outcome", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Dataframe to Tensor

In [157]:
X_train_tensor = torch.tensor(X_train.values).cuda()
X_test_tensor = torch.tensor(X_test.values).cuda()
y_train_tensor = torch.tensor(y_train.values).cuda()
y_test_tensor = torch.tensor(y_test.values).cuda()
print(X_train_tensor.shape)
print(X_test_tensor.shape)
print(y_train_tensor.shape)
print(y_test_tensor.shape)

X_train_tensor = X_train_tensor.to(torch.float32)
X_test_tensor = X_test_tensor.to(torch.float32)

print(X_train_tensor.dtype)
print(X_test_tensor.dtype)
print(y_train_tensor.dtype)
print(y_test_tensor.dtype)

torch.Size([537, 8])
torch.Size([231, 8])
torch.Size([537])
torch.Size([231])
torch.float32
torch.float32
torch.int64
torch.int64


Create a neural network using PyTorch

In [158]:
class ShallowNeuralNetwork(torch.nn.Module): 
    def __init__(self, input_size, output_size, hidden_size):
        super(ShallowNeuralNetwork, self).__init__()
        # Layer to use
        self.linear1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(hidden_size, output_size)
    
    def forward(self, x): 
        out1 = self.linear1(x)
        out2 = self.relu(out1)
        out3 = self.linear2(out2)
        return out3

In [159]:
input_size = 8
output_size = 1
hidden_size = 10
shallow_model = ShallowNeuralNetwork(input_size, output_size, hidden_size).to(device)
print(shallow_model)

ShallowNeuralNetwork(
  (linear1): Linear(in_features=8, out_features=10, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=10, out_features=1, bias=True)
)


Training

In [160]:
num_epochs = 120
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(shallow_model.parameters(), lr = 1e-3)

for epoch in range(num_epochs): 
    shallow_model.train()
    epoch += 1
    # clear previous gradients
    optimizer.zero_grad()
    # forward pass
    pred = shallow_model(X_train_tensor).squeeze()
    # calculate loss
    loss = criterion(pred, y_train_tensor)
    # backward pass 
    loss.backward()
    # optimization
    optimizer.step()
    # prediction 
    prediction = torch.round(torch.sigmoid(pred))
    accuracy = (prediction==y_train_tensor).sum().item()/len(y_train_tensor)

    shallow_model.eval()
    test_pred = shallow_model(X_test_tensor).squeeze()
    test_loss = criterion(test_pred , y_test_tensor)
    # prediction
    test_prediction = torch.round(torch.sigmoid(test_pred))
    test_accuracy = (test_prediction== y_test_tensor).sum().item()/len(y_test_tensor)

    if epoch % 20 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss}, Test Loss: {test_loss}, Accuracy: {accuracy}, Test Accuracy: {test_accuracy}')

RuntimeError: Found dtype Long but expected Float